<a href="https://colab.research.google.com/github/kim-juhyeon/Practical-Time-Series-Analysis/blob/master/Ch02/2%EC%9E%A5_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd


### 회원의 전체 기부금과 열람 이메일의 전체 개수와 , 어떤 상관관계가 있는지 알아보자
>회원이 가입 직후에 지불할 것인지 아니면 시간이 꽤 흐른 뒤 기부할 것인지 같은 시간 중심적인 질문에도 답할 수 있습니다. 
>> 하지만 누가 언제 기부 할지 예측하기에는 어렵다


In [3]:
emails = pd.read_csv('/content/drive/MyDrive/BookRepo/Ch02/data/emails.csv')
emails
# week <- 매주의 기간에 대한 기록임을 암시한다. 하지만 데이터가 한 주 동안의 집계라는 것은 명백한 사실.
# 사람의 활동 분석을 할 경우에는 1월1일 기준으로 알아보기 보다는 월~일요일로 정하는게 좋습니다.

,user,week,emailsOpened
0,0.0,2018-04-02 00:00:00,2.0
1,0.0,2018-04-09 00:00:00,2.0
2,0.0,2018-04-16 00:00:00,1.0
3,0.0,2018-04-23 00:00:00,2.0
4,1.0,2017-10-09 00:00:00,3.0
...,...,...,...
28286,999.0,2018-04-23 00:00:00,3.0
28287,999.0,2018-04-30 00:00:00,3.0
28288,999.0,2018-05-07 00:00:00,3.0
28289,999.0,2018-05-14 00:00:00,3.0


In [4]:
donations = pd.read_csv("/content/drive/MyDrive/BookRepo/Ch02/data/donations.csv")
donations

,amount,timestamp,user
0,25.0,2017-11-12 11:13:44,0.0
1,50.0,2015-08-25 19:01:45,0.0
2,25.0,2015-03-26 12:03:47,0.0
3,50.0,2016-07-06 12:24:55,0.0
4,50.0,2016-05-11 18:13:04,1.0
...,...,...,...
2671,25.0,2016-09-02 11:20:00,992.0
2672,50.0,2017-11-02 12:17:06,993.0
2673,1000.0,2016-09-13 21:09:47,995.0
2674,1000.0,2017-09-29 20:03:01,995.0


In [5]:
YearJoined = pd.read_csv("/content/drive/MyDrive/BookRepo/Ch02/data/year_joined.csv")
YearJoined

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018
...,...,...,...
995,995,bronze,2016
996,996,bronze,2018
997,997,bronze,2018
998,998,bronze,2017


### 보유한 데이터에서 시간축을 고려하는 것부터 천천히 알아보자
- 연간 회원 상태
- 이메일 열람에 대한 주간 누적 기록
- 기부가 이뤄진 순간의 타임스탬프

In [6]:
YearJoined.groupby('user').count().groupby('userStats').count()
# 위 코드는 1,000명의 모든 회원이 단 하나의 상태만 가진다는 사실을 보여준다.<- 중복된 유저의 state가 없으며 , 1000개로 각기 나눠저 있음
# 데이터를 좀 더 깊게 분석하려면 데이터 파이프라인을 아는 사람과 함께 명확히 알고 있어야 한다. <- 사전관찰

,yearJoined
userStats,
1,1000


In [7]:
emails[emails.emailsOpened < 1]
# 여기에 널이 아에예 발생하지 않았거나 , 모든 회원이 이메일을 열람했다는 이벤트가 적어도 하나 존재한다는 두 가지 가능성이 있습니다.
# <-이메일을 열람하도록 사람을 유도하는 것은 굉장히 어렵다. 그러므로 매주 적어도 한번 이상 이메일을 확인 할 가능성이 매우 낮을 것으로 예상하고, 특정 한 회원에 대한 기록을 살펴봄으로써 둘 중 무엇인지 판단해보자

,user,week,emailsOpened


In [8]:
#emails[emailsOpened < 1]
# emails안에 emailsOpened가 있으므로 "."을 이용하고 그 전에는 emails 속성이므로 앞에 쓴다.

In [9]:
emails[emails.user == 997] # 코드 결과에 따라 일부 주가 누락된 것을 확인 할 있다. 특정 회원에 대한 사건이 처음 발생한 시점과 마지막 시점을 사이에 주가 몇개 기록 되었지를 계산하면, 좀 더 수학적인 방식으로 검증할 수 있음

,user,week,emailsOpened
28110,997.0,2017-06-19 00:00:00,1.0
28111,997.0,2017-07-03 00:00:00,1.0
28112,997.0,2017-07-31 00:00:00,2.0
28113,997.0,2017-08-14 00:00:00,1.0
28114,997.0,2017-08-28 00:00:00,1.0
28115,997.0,2017-09-04 00:00:00,2.0
28116,997.0,2017-09-11 00:00:00,1.0
28117,997.0,2017-09-18 00:00:00,3.0
28118,997.0,2017-09-25 00:00:00,2.0
28119,997.0,2017-10-02 00:00:00,1.0


In [10]:
import datetime
date_time_str_max = max(emails[emails.user == 997].week)
date_time_str_min = min(emails[emails.user == 997].week)

date_time_obj_max = datetime.datetime.strptime(date_time_str_max, '%Y-%m-%d %H:%M:%S')
date_time_obj_min = datetime.datetime.strptime(date_time_str_min, '%Y-%m-%d %H:%M:%S')

(date_time_obj_max - date_time_obj_min).days/7

49.0

In [11]:
print(date_time_str_max)
# strptime()메서드 는 지정된 문자열에서 날짜/시간 개체를 만듭니다.
print(date_time_obj_min)
print(date_time_obj_max)
print(date_time_obj_max - date_time_obj_min) # <- 343기간안에 열람을 했음 이를 주로 나누면 49주이다.

2018-05-28 00:00:00
2017-06-19 00:00:00
2018-05-28 00:00:00
343 days, 0:00:00


In [12]:
emails[emails.user == 997].shape # 해당 회원의 주간 데이터는 40행이 있다. 하지만 모든 주가 포함된 데이터라면 실제로 41개가 있어야 합니다.
# <- 왜 41개 일까? 시계열 데이터 작업시 이런 종류의 뺄셈을 해야한다면 1을 더해야 하는지에 대해 항상 의문이다, 즉, 계산에 반영되어야 하는 위치를 빼는 것이 맞는지 확인해야 합니다.(처음과 마지막 날의 차이를 7로 나눈 다음, 누락된 시작 날짜를 위한 1을 더해주는 것이 적절한 계산법)

(40, 3)

In [13]:
##누락된 주가 있다는 사실을 확인했다면, 그 부분을 채워 넣어보자.
# 팬더스 데이터프레임 다중 색인을 위한 멀티인덱스 <- 곱집합을 수행합니다.
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))

In [14]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value=0).reset_index()
all_email.columns = ['week', 'user', 'emailsOpened']
all_email[all_email.user == 997].sort_values('week') #오름차순 (sort)
#complete_idx로 재색인하여 누락된 값을 채워 넣습니다. 이때 기록할 것이 없다는 의미로 누락된 값은 0으로 채웁니다. <- 재설정하여 색인 할 경우 회원과 주의 정보를 다시 열로 만드는데 이때 열의 정보가 유실 될 수 있으므로 명시적으로 붙여줍니다.
# 첫 이메일오픈한 데이터는 0이 많다 <- 회원가입 전단계여서 이메일이 포함이 안되어 있음을 유추(비회원)

,week,user,emailsOpened
52738,2015-02-09 00:00:00,997.0,0.0
73834,2015-02-16 00:00:00,997.0,0.0
11718,2015-02-23 00:00:00,997.0,0.0
73248,2015-03-02 00:00:00,997.0,0.0
36916,2015-03-09 00:00:00,997.0,0.0
...,...,...,...
8202,2018-04-30 00:00:00,997.0,3.0
91414,2018-05-07 00:00:00,997.0,3.0
80280,2018-05-14 00:00:00,997.0,3.0
18750,2018-05-21 00:00:00,997.0,3.0


In [15]:
all_email.shape

(101378, 3)

In [16]:
#이메일 데이터프레임을 회원별로 그룹화하여 start_date , end_date 를 결정하고, 이로부터 각 회원이 이메일을 수신해온 "주"의 시작, 끝을 얻을 수 있습니다.
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates = cutoff_dates.reset_index()

In [17]:
cutoff_dates # agg 는 집계 의 별칭입니다 . 별칭을 사용합니다.

,index,user,min,max
0,0,0.0,2018-04-02 00:00:00,2018-04-23 00:00:00
1,1,1.0,2017-10-09 00:00:00,2018-05-28 00:00:00
2,2,2.0,2016-11-14 00:00:00,2018-05-28 00:00:00
3,3,4.0,2017-08-21 00:00:00,2017-11-13 00:00:00
4,4,6.0,2017-06-19 00:00:00,2017-08-21 00:00:00
...,...,...,...,...
581,581,991.0,2017-12-18 00:00:00,2018-05-28 00:00:00
582,582,992.0,2017-03-20 00:00:00,2018-03-05 00:00:00
583,583,996.0,2015-03-02 00:00:00,2015-04-06 00:00:00
584,584,997.0,2017-06-19 00:00:00,2018-05-28 00:00:00


In [18]:
import warnings
warnings.filterwarnings('ignore')

for _, row in cutoff_dates.iterrows():
  user        = row['user']
  start_date  = row['min']
  end_date    = row['max']
  all_email.drop(all_email[all_email.user == user][all_email.week < start_date].index, inplace=True)
  all_email.drop(all_email[all_email.user == user][all_email.week > end_date].index, inplace=True)

In [19]:
all_email[all_email.user == 997].sort_values('week')
all_email.shape

(37300, 3)

In [23]:
# all_email 멀티인덱스를 이용해서 빈곳을 채운 수가 ->(101378, 3) 였으며, max,min 값을 이용해 0이였던 기여가 별로 없는 행을 삭제했습니다. ->(37300, 3)

(50, 3)

In [26]:
all_email[all_email.user == 997].sort_values('week') # 이렇게 진행하면 다른 회원과 동일한 날짜에 빈곳이 채워지고 (기존에 없었다면 = 0) max값 이후는 삭제가 됌

,week,user,emailsOpened
58012,2017-06-19 00:00:00,997.0,1.0
94344,2017-06-26 00:00:00,997.0,0.0
66216,2017-07-03 00:00:00,997.0,1.0
99032,2017-07-10 00:00:00,997.0,0.0
38088,2017-07-17 00:00:00,997.0,0.0
79108,2017-07-24 00:00:00,997.0,0.0
39260,2017-07-31 00:00:00,997.0,2.0
62114,2017-08-07 00:00:00,997.0,0.0
79694,2017-08-14 00:00:00,997.0,1.0
22266,2017-08-21 00:00:00,997.0,0.0


### 이메일과 기부 데이터를 어떻게 서로 연관 지어 볼까?
- 기부 데이터를 다운샘플링하여 주간 시계열로 바꿀 수 있다.
- 사실상 주간 기부금은 기부자 개별의 기부 금액이 반영된 경우가 대부분입니다.(한 주에 두 번 이상 기분하는 사람이 있을 가능성이 낮기 때문

In [27]:
donations.timestamp = pd.to_datetime(donations.timestamp)
donations.set_index('timestamp', inplace=True)
agg_don = donations.groupby('user').apply(lambda df: df.amount.resample("W-MON").sum().dropna())
agg_don

user   timestamp 
0.0    2015-03-30      25.0
       2015-04-06       0.0
       2015-04-13       0.0
       2015-04-20       0.0
       2015-04-27       0.0
                      ...  
995.0  2017-09-11       0.0
       2017-09-18       0.0
       2017-09-25       0.0
       2017-10-02    1000.0
998.0  2018-01-08      50.0
Name: amount, Length: 32352, dtype: float64